In [4]:
# Gerekli olan kütüpanelerin import edilmesi.

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import neighbors
from sklearn.svm import SVR

In [2]:
# Bazı algoritmaları çalıştırırken hata olarak görünüyor fakat çalışıyor bu sorunun önüne geçemk için aşağıdaki işlemi gerçekleştiriyorum.

from warnings import filterwarnings
filterwarnings('ignore')

In [23]:
# Veri setinin okunmasını sağlıyorum.
df = pd.read_csv(r"C:\Users\realb\Desktop\ML_dersler\ML_301\Yapay Sinir Ağlar\Hitters.csv")

# Veri setinin içinden eksik gözlemleri çıkarmak gerekiyor bu işlemi yapıyorum.
df = df.dropna()

# Şimdi ise dummies ile kukla veri işlemi yapıyorum. One end coding yöntemi ile bu işlemi yapıyorum.
dms = pd.get_dummies(df[['League', 'Division','NewLeague']])

# Bağımlı değişkenimi "y" atıyorum.
y = df["Salary"]

# Veri setimiz içindeki bağımlı değişkeni ve katogorik değişkenlerin götürülmesi işlemini yapıyorum.
X_ = df.drop(['Salary', 'League', 'Division', 'NewLeague'], axis=1).astype('float64')

# Sonuçunda yukardaki değerlerin dummi versyonlarını ve bağımsız değişkenlerimi bir arada tutma işlemini yapıyorum.
X = pd.concat([X_, dms[['League_N', 'Division_W', 'NewLeague_N']]], axis=1)

# Şimdi ise veri setimizde train ve test ayrımı yapmak için bu ayrımı yapma işlemini yapıyorum.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [26]:
# yapay sinir ağları ile model geliştirirken standartlaştırma yapmamız başarı yüzdesini artırmaktadır o yüzden şimdi bu işlemi yapıyorum.
scaler = StandardScaler()

In [27]:
# Şimdi burda benim yaptığım elimde olan x train veri setini kulanarak veri setini standartlaştırma işlemini yapmaktayım.
scaler.fit(X_train)
# X_train ölçeklendirmasini yapıyorum.
X_train_scaled = scaler.transform(X_train)

In [29]:
# Benzer işlemi tast veri setimiz içinde gerçekleştirmek gerekiyor.
X_test_scaled = scaler.transform(X_test)

In [30]:
# Model kurlumunu gerçekleştirmeye başlıyorum, modelin fit ederken x değişkenin standartlaşmaya ihtiyacı var dışarda bağımlı değişkeinin değil.
cnn_model = MLPRegressor().fit(X_train_scaled, y_train)

In [35]:
help(cnn_model)

Help on MLPRegressor in module sklearn.neural_network._multilayer_perceptron object:

class MLPRegressor(sklearn.base.RegressorMixin, BaseMultilayerPerceptron)
 |  MLPRegressor(hidden_layer_sizes=(100,), activation='relu', *, solver='adam', alpha=0.0001, batch_size='auto', learning_rate='constant', learning_rate_init=0.001, power_t=0.5, max_iter=200, shuffle=True, random_state=None, tol=0.0001, verbose=False, warm_start=False, momentum=0.9, nesterovs_momentum=True, early_stopping=False, validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08, n_iter_no_change=10, max_fun=15000)
 |  
 |  Multi-layer Perceptron regressor.
 |  
 |  This model optimizes the squared error using LBFGS or stochastic gradient
 |  descent.
 |  
 |  .. versionadded:: 0.18
 |  
 |  Parameters
 |  ----------
 |  hidden_layer_sizes : array-like of shape(n_layers - 2,), default=(100,)
 |      The ith element represents the number of neurons in the ith
 |      hidden layer.
 |  
 |  activation : {'identity',

In [68]:
# Şimdi indergenen veri gurubu içinden verileri kulanarak tahmin etme işlemi gerçekleştirelim.
cnn_model.predict(X_test_scaled)[0:5]

array([18.87723153, 39.79526013, 69.03632145,  9.38909343, 20.47700953])

In [70]:
# Test hatsı RMSE değeri için bir hesaplama işlemi yapıyorum.
y_pred = cnn_model.predict(X_test_scaled)
np.sqrt(mean_squared_error(y_pred, y_test))


660.8104485423368

In [47]:
# şimid burda alpha ve hidden layer ile ilgili paremetreleri denenmesini yapaçağım.
# Buraya girerken alpha değerleini normal olarak girişini yapıyorum Yani almasını istediğim alpha değerlerini yazarak tanımlıyorum.
# Hiddin layer değerinin tanımlamasını yaparken ise iki değerler olarak tanımlamasını yapıyorum ve burda 1. değer bu kadar hiden katmanına bu kadar hüzre yapısı anlamana gelmektedir.
cnn_params = {"alpha": [0.1, 0.01, 0.02, 0.03, 0.001, 0.0001],
              "hidden_layer_sizes": [(10,20), (5,5), (100,100)]}


In [56]:
# Şimdi ben buraya cros validation işlemi yapacak şekilde model tanımlamasını yapıyorum.
cnn_cv_model = GridSearchCV(cnn_model, cnn_params, cv=20, verbose=2, n_jobs=-1).fit(X_train_scaled, y_train)

Fitting 20 folds for each of 18 candidates, totalling 360 fits


In [63]:
# Şimdi burdan en iyi hangi hiperparemetre daha başarılı ona karar veriyoruz.

cnn_cv_model.best_params_


{'alpha': 0.01, 'hidden_layer_sizes': (100, 100)}

Final model

In [64]:
# En iyi sonucu veren hiperparemetre değerleri ile tunned edilmiş olan modelin kurulumunu yapıyorum.
cnn_tunned = MLPRegressor(alpha=0.01, hidden_layer_sizes = (100,100)).fit(X_train_scaled, y_train)

In [65]:
y_pred = cnn_tunned.predict(X_test_scaled)

In [66]:
# Tunned işlemi yapılmış modelim hata kat sayı RMSE değerine bakıyoruz.
np.sqrt(mean_squared_error(y_test, y_pred))

362.6305722920098